In [1]:
from datetime import date
from data_manager import DataManager
from scipy.stats import median_abs_deviation
import pandas as pd


In [2]:
self = DataManager()

def get_average_team_stat(self, df, last_n_team_games):
    teams = self.query_teams()
    
    # Initialize columns outside the loop
    stats = ['fgm', 'fga', 'fg3m', 'fg3a', 'points', 'rebounds', 'assists', 'steals', 'blocks', "to"]
    for stat in stats:
        df.loc[:, f'team_{stat}_mad'] = None
        df.loc[:, f'team_{stat}_average'] = None

    for team in df['TEAM'].unique():
        team_id = [team_object.id for team_object in teams if team_object.nickname == team][0]
        print(team_id)
        team_games = self.get_and_save_team_data(team_id).sort_values(by='date', ascending=False).head(last_n_team_games)
        print(team_games.columns)
        for stat in stats:
            total_points_average = team_games[stat].mean()
            mad_points = median_abs_deviation(team_games[stat], scale="normal")
            df.loc[df['TEAM'] == team, f'team_{stat}_average'] = total_points_average
            df.loc[df['TEAM'] == team, f'team_{stat}_mad'] = mad_points
            
    return df

In [3]:

analyzed_props = self.get_analyzed_props()
analyzed_prop_records = pd.DataFrame.from_dict([prop.entry for prop in analyzed_props])
profitable_props = analyzed_prop_records[analyzed_prop_records['EV'] > 0]


Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
Loading steals props.
Loading blocks props.
           player_name    team    stat  over_threshold  over_odds  \
0           Obi Toppin  Pacers  points             8.5       -105   
1        Pascal Siakam  Pacers  points            21.5       -120   
2        Aaron Nesmith  Pacers  points             9.5       -110   
3    Tyrese Haliburton  Pacers  points            18.5       -115   
4       T.J. McConnell  Pacers  points            10.5        105   
..                 ...     ...     ...             ...        ...   
84           Josh Hart  Knicks  blocks             0.5       -115   
85    Donte DiVincenzo  Knicks  blocks             0.5       -130   
86  Isaiah Hartenstein  Knicks  blocks             0.5       -230   
87       Miles McBride  Knicks  blocks             0.5        275   
88       Jalen Brunson  Knicks  blocks             0.5        350   


In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
view = profitable_props[['PLAYER', 'TEAM', 'STAT', 'TYPE']]
last_n_team_games = 25
df = get_average_team_stat(self, profitable_props, last_n_team_games=last_n_team_games)

18
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date'],
      dtype='object')
16
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date'],
      dtype='object')


C:\Users\rusta\AppData\Local\Temp\ipykernel_27636\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_mad'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_27636\4244349758.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_average'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_27636\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [5]:
print(df) 

df = df.sort_values(by="EV", ascending=False).reset_index(drop=True)
df.to_csv("temp.csv")
print(df.columns)


                 PLAYER    TEAM      STAT  THRESH  ODDS   TYPE      PROB  \
0            Obi Toppin  Pacers    points     8.5  -105   over  0.781912   
4         Aaron Nesmith  Pacers    points     9.5  -110   over  0.609900   
8        T.J. McConnell  Pacers    points    10.5   105   over  0.723500   
10      Andrew Nembhard  Pacers    points     9.5  -125   over  0.684958   
12         Myles Turner  Pacers    points    15.5  -125   over  0.610214   
14         Ben Sheppard  Pacers    points     4.5  -110   over  0.657850   
16   Isaiah Hartenstein  Knicks    points     8.5  -105   over  0.714490   
19        Jalen Brunson  Knicks    points    35.5  -125  under  0.603772   
20     Donte DiVincenzo  Knicks    points    17.5  -110   over  0.635333   
22            Josh Hart  Knicks    points    11.5  -125   over  0.662870   
25        Miles McBride  Knicks    points    13.5  -110  under  0.599410   
26    Tyrese Haliburton  Pacers   assists     8.5  -105   over  0.523087   
28       T.J

In [6]:
from collections import deque

teams = df['TEAM'].unique()
if len(teams) != 2:
    raise ValueError("The TEAM column must contain exactly two unique values for this operation.")

# Get the values to be swapped
value1 = df.loc[df['TEAM'] == teams[0], 'team_to_average'].values[0]
value2 = df.loc[df['TEAM'] == teams[1], 'team_to_average'].values[0]

# Create a mapping to swap the values
mapping = {teams[0]: value2, teams[1]: value1}

# Apply the mapping to the 'team_average_to' column
df['team_to_average'] = df['TEAM'].map(mapping)

# Rename the column
df.rename(columns={'team_to_average': 'oppteam_to_average'}, inplace=True)

    

In [7]:
team_stats_cols = [col for col in df.columns if "average" in col]
team_stats = df.copy()[team_stats_cols]
new_cols = [col.split("_")[1] for col in team_stats.columns]
team_stats.columns = new_cols
team_stats['TEAM'] = df['TEAM']


initial_availability = {}
for team in team_stats['TEAM'].unique():
    initial_availability[team] = dict(team_stats[team_stats['TEAM'] == team].reset_index(drop=True).iloc[0, :])



                

In [9]:

import copy

parlay_length = 9
parlay_count = 5
used = []
parlays = []
for i in range(parlay_count):
    availability = copy.deepcopy(initial_availability)
    parlay = []
    counter = 0
    for i, row in df.iterrows():
        if len(parlay) == parlay_length:
            break
        if i in used:
            continue
        else:
            if row['STAT'] in new_cols:
                stat = row['STAT']
                if row['TYPE'] == "over" and availability[row['TEAM']][stat] - (row['THRESH'] * 3) > 0:
                    availability[row['TEAM']][stat] -= row['THRESH']
                    parlay.append(i)
                    used.append(i)
                elif row['TYPE'] == "under" and availability[row['TEAM']][stat] + row['THRESH'] < initial_availability[row['TEAM']][stat]:
                    availability[row['TEAM']][stat] += row['THRESH']
                    parlay.append(i)
                    used.append(i)
    print(parlay)
    parlays.append(parlay)
parlays = [parlay for parlay in parlays if len(parlay) == parlay_length]
print(parlays)


                    

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 16, 19, 22, 23, 25]
[28, 29, 30, 32, 33, 34, 35, 37, 41]
[44, 47, 48, 52, 53, 56, 58, 59, 60]
[63, 64]
[[1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 16, 19, 22, 23, 25], [28, 29, 30, 32, 33, 34, 35, 37, 41], [44, 47, 48, 52, 53, 56, 58, 59, 60]]


In [10]:
parlay_dfs = []
# Segment the original DataFrame
for indexes in parlays:
    parlay_df = df.iloc[indexes]
    parlay_dfs.append(parlay_df)

# Display the resulting DataFrames
for i, segment_df in enumerate(parlay_dfs):
    print(f"Parlay {i}:\n{segment_df}\n")
    segment_df.to_csv(f"parlays/{date.today()}_{i}_greedy.csv")

Parlay 0:
               PLAYER    TEAM     STAT  THRESH  ODDS  TYPE      PROB  \
1          Obi Toppin  Pacers   points     8.5  -105  over  0.781912   
2      T.J. McConnell  Pacers   points    10.5   105  over  0.723500   
3      T.J. McConnell  Pacers  assists     4.5   110  over  0.681536   
4          Obi Toppin  Pacers      fgm     3.5   125  over  0.633536   
5  Isaiah Hartenstein  Knicks   points     8.5  -105  over  0.714490   
6      T.J. McConnell  Pacers      fgm     4.5   100  over  0.687282   
7        Ben Sheppard  Pacers  assists     0.5  -120  over  0.698806   
8        Ben Sheppard  Pacers   points     4.5  -110  over  0.657850   
9           Josh Hart  Knicks  assists     3.5  -170  over  0.786709   

         EV  HOUSE_PROB team_fgm_mad team_fgm_average team_fga_mad  \
1  2.632955    0.512195     5.930409            46.52     4.447807   
2  2.415878    0.487805     5.930409            46.52     4.447807   
3  2.156128    0.476190     5.930409            46.52     4